In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import torch

import sys
# import svgutils.transform as sg
# from svgutils.compose import *

import plotting_utils
from fnope.utils.misc import get_output_dir

In [ ]:
#### Load and prepare data ####
out_dir = get_output_dir()
# Change the value below for the different experiments of ice - using 50 parameters as in Moss et al., or all 500
num_params = 50
if num_params == 500:
    experiment_folder = out_dir/"ice_experiment_500"
    method_names = ["FNOPE", "NPE (spectral)", "FMPE (spectral)", "NPE (raw)"] #FMPE raw fails for 500 params
    custom_order = [1, 2, 3, 4]  # New order of methods_names
    upper_lim = 0.8


else:
    experiment_folder = out_dir/"ice_experiment"
    method_names = ["FNOPE", "NPE (spectral)", "FMPE (raw)", "FMPE (spectral)", "NPE (raw)"]
    custom_order = [1, 2, 4, 5, 3]  # New order of methods_names
    upper_lim = 0.7


path = experiment_folder/ "summary.csv"
data = pd.read_csv(path)


# Take nsim= 100 from the rerun data and append it to the original data

data


# Get the different methods
methods = data["method"].unique()
n_sim = data["nsim"].unique()

# Calculate mean and SE for each method and each number of simulations
mses_results = {}
mses_real_results = {}
sbc_results = {}
mses_mean = np.zeros((methods.shape[0], n_sim.shape[0]))
mses_SE = np.zeros((methods.shape[0], n_sim.shape[0]))
mses_real_mean = np.zeros((methods.shape[0], n_sim.shape[0]))
mses_real_SE = np.zeros((methods.shape[0], n_sim.shape[0]))
sbcs_mean = np.zeros((methods.shape[0], n_sim.shape[0]))
sbcs_SE = np.zeros((methods.shape[0], n_sim.shape[0]))

for ii, method in enumerate(methods):
    mses_results[method] = {}
    mses_real_results[method] = {}
    sbc_results[method] = {}
    for kk, nsim in enumerate(n_sim):
        mses_results[method][nsim] = []
        mses_real_results[method][nsim] = []
        sbc_results[method][nsim] = []
        temp_mses = data[(data["method"] == method) & (data["nsim"] == nsim)][
            "predictive_mses"
        ]
        temp_mses_real = data[(data["method"] == method) & (data["nsim"] == nsim)][
            "predictive_mses_real_data"
        ]
        temp_sbcs = data[(data["method"] == method) & (data["nsim"] == nsim)]["sbcs"]

        for ll in range(temp_mses.shape[0]):
            s_mses = temp_mses.iloc[ll]
            s_mses_clean = s_mses.replace("[", "").replace("]", "")
            list_mses = [float(x) for x in s_mses_clean.split() if x != "nan"]
            mses_results[method][nsim].extend(list_mses)

        mses_mean[ii, kk] = np.mean(np.array(mses_results[method][nsim]))
        mses_SE[ii, kk] = np.std(np.array(mses_results[method][nsim])) / np.sqrt(
            len(mses_results[method][nsim])
        )

        for mm in range(temp_sbcs.shape[0]):
            s_sbcs = temp_sbcs.iloc[mm]
            s_sbcs_clean = s_sbcs.replace("[", "").replace("]", "")
            list_sbcs = [float(x) for x in s_sbcs_clean.split() if x != "nan"]
            sbc_results[method][nsim].extend(list_sbcs)

        sbcs_mean[ii, kk] = np.mean(np.array(sbc_results[method][nsim]))
        sbcs_SE[ii, kk] = np.std(np.array(sbc_results[method][nsim])) / np.sqrt(
            len(sbc_results[method][nsim])
        )

        for ll in range(temp_mses_real.shape[0]):
            s_mses_real = temp_mses_real.iloc[ll]
            s_mses_real_clean = s_mses_real.replace("[", "").replace("]", "")
            list_mses_real = [float(x) for x in s_mses_real_clean.split() if x != "nan"]
            mses_real_results[method][nsim].extend(list_mses_real)

        mses_real_mean[ii, kk] = np.mean(np.array(mses_real_results[method][nsim]))
        mses_real_SE[ii, kk] = np.std(
            np.array(mses_real_results[method][nsim])
        ) / np.sqrt(len(mses_real_results[method][nsim]))

In [ ]:
data

In [ ]:
#### Calculate ideal SBC to add to SBC plot ####

num_posterior_samples = 1000
n_sbc = 100
n_sbc_marginals = 50

ranks = np.random.randint(0, num_posterior_samples, size=(n_sbc, n_sbc_marginals))

coverage_values = torch.Tensor(ranks) / num_posterior_samples

absolute_atcs = []

for dim_idx in range(coverage_values.shape[1]):
    # calculate empirical CDF via cumsum and normalize
    hist, alpha_grid = torch.histogram(
        coverage_values[:, dim_idx], density=True, bins=30
    )
    # add 0 to the beginning of the ecp curve to match the alpha grid
    ecp = torch.cat([torch.Tensor([0]), torch.cumsum(hist, dim=0) / hist.sum()])
    absolute_atc = (ecp - alpha_grid).abs().mean().item()
    absolute_atcs.append(absolute_atc)

absolute_atcs = torch.tensor(absolute_atcs)
mean_absolute_atc = absolute_atcs.mean().numpy()
print(mean_absolute_atc)

In [ ]:
print(n_sim)

In [ ]:
#### Create three plots for these metrics ####

with plt.rc_context(fname="matplotlibrc"):

    plt.tight_layout()
    # fig, axs = plt.subplots(1, 3, figsize=(16, 3))
    #
    #fig, axs = plt.subplots(1, 3, figsize=(8, 1.3))
    fig, axs = plt.subplots(1, 3, figsize=(5.5206, 0.9))

    fig.subplots_adjust(wspace=0.7)

    # colors = ["#9b2226", "#023e8a", "#00b4d8", "#0077b6", "#90e0ef"]
    # colors1 = ["#9b2226", "#023e8a", "#00b4d8", "#0077b6", "#90e0ef"]
    # colors2 = ["#9b2226", "#023e8a", "#00b4d8", "#0077b6", "#90e0ef"]
    colors = [plotting_utils.colors[plotting_utils.method_names.index(method_names[i])] for i in range(len(method_names))]

    for mm in range(methods.shape[0]):

        axs[0].errorbar(
            n_sim,
            mses_mean[mm, :],
            yerr=mses_SE[mm, :],
            fmt="o",
            linestyle="-",
            color=colors[mm],
            label=method_names[mm],
        )
        axs[1].errorbar(
            n_sim,
            sbcs_mean[mm, :],
            yerr=sbcs_SE[mm, :],
            fmt="o",
            linestyle="-",
            color=colors[mm],
            label=method_names[mm],
        )
        axs[2].errorbar(
            n_sim,
            mses_real_mean[mm, :],
            yerr=mses_real_SE[mm, :],
            fmt="o",
            linestyle="-",
            color=colors[mm],
            label=method_names[mm],
        )

    axs[0].set_xscale("log")
    axs[0].set_xlabel("# simulations")
    axs[0].set_ylabel("MSE to synth. obs.")
    axs[0].set_xticks(n_sim)
    axs[0].set_yticks([0, 0.6])
    axs[0].set_ylim([0, upper_lim])
    axs[0].minorticks_off()
    axs[0].spines["left"].set_position(("outward", 5))  # Move y-axis slightly left
    axs[0].spines["bottom"].set_position(("outward", 5))

    axs[1].hlines(
        mean_absolute_atc,
        1e2,
        1e5,
        linestyle=":",
        linewidth=2.5,
        color="black",
        label="lower bound",
    )
    axs[1].set_xscale("log")
    axs[1].set_xlabel("# simulations")
    axs[1].set_ylabel("SBC EoD")
    axs[1].set_xticks(n_sim)
    axs[1].set_ylim([0, 0.2])
    axs[1].set_yticks([0, 0.2])
    axs[1].minorticks_off()
    axs[1].spines["left"].set_position(("outward", 5))  # Move y-axis slightly left
    axs[1].spines["bottom"].set_position(("outward", 5))

    # Create the legend with the new order
    # axs[1].legend(handles=handles,
    # labels=labels,
    # loc="upper center",
    # bbox_to_anchor=(0.5, -0.6),
    # ncol=5
    # )

    axs[2].set_xscale("log")
    axs[2].set_xlabel("# simulations")
    axs[2].set_ylabel("MSE to real obs.")
    axs[2].set_xticks(n_sim)
    axs[2].set_yticks([0, 0.6])
    axs[2].set_ylim([0, upper_lim])
    axs[2].minorticks_off()
    axs[2].spines["left"].set_position(("outward", 5))  # Move y-axis slightly left
    axs[2].spines["bottom"].set_position(("outward", 5))

    # Add legends
    # Get handles and labels
    handles, labels = axs[1].get_legend_handles_labels()


    # Reorder handles and labels based on custom order
    handles_global = [handles[i] for i in custom_order]
    labels_global = [labels[i] for i in custom_order]

    handle_ideal = [handles[0]]
    label_ideal = [labels[0]]

    axs[1].legend(
        handles=handle_ideal,
        labels=label_ideal,
        loc="upper right",
        bbox_to_anchor=(1.1, 1.1),
        handlelength=1.5,
        frameon=False,
    )

    # Create the legend with the new order
    fig.legend(
        handles=handles_global,
        labels=labels_global,
        loc="upper center",
        bbox_to_anchor=(0.5, -0.45),
        ncol=5,
    )
    # if num_params == 500:
    #     plt.savefig(
    #         "ice_plots/results_ice_500.pdf", format="pdf", bbox_inches="tight"
    #     )
    # else:
    #     plt.savefig("ice_plots/results_ice.svg", format="svg", bbox_inches="tight")
    plt.show()

In [ ]:
def mse_markdown_table_with_se(mses_results, mses_SE, n_sim, method_names_map):
    method_names = list(mses_results.keys())
    sim_counts = list(n_sim)
    header = "| Method / # simulations | " + " | ".join([f"{s:,}" for s in sim_counts]) + " |"
    sep = "|" + "---|" * (len(sim_counts) + 1)
    rows = []
    for idx, method in enumerate(method_names):
        display_name = method_names_map.get(method, method)
        means = [np.mean(mses_results[method][sim]) for sim in sim_counts]
        ses = [mses_SE[idx, k] for k in range(len(sim_counts))]
        row = "| " + display_name + " | " + " | ".join(
            [f"{m:.3f} ± {se:.3f}" for m, se in zip(means, ses)]
        ) + " |"
        rows.append(row)
    return "\n".join([header, sep] + rows)

method_names_map = {
    "FNOPE_always_equispaced_False": "FNOPE",
    "spectral_NPE": "NPE (spectral)",
    "raw_FMPE": "FMPE (raw)",
    "raw_NPE": "NPE (raw)",
    "spectral_FMPE": "FMPE (spectral)"
}

print(mse_markdown_table_with_se(mses_results, mses_SE, n_sim, method_names_map))

| Method / # simulations | 100 | 1,000 | 10,000 | 100,000 |
|---|---|---|---|---|
| FNOPE | 0.108 ± 0.002 | 0.069 ± 0.002 | 0.054 ± 0.001 | 0.046 ± 0.001 |
| NPE (spectral) | 0.355 ± 0.006 | 0.180 ± 0.003 | 0.071 ± 0.002 | 0.062 ± 0.001 |
| FMPE (raw) | 0.244 ± 0.003 | 0.238 ± 0.003 | 0.074 ± 0.001 | 0.058 ± 0.002 |
| NPE (raw) | 0.246 ± 0.003 | 0.287 ± 0.004 | 0.093 ± 0.002 | 0.045 ± 0.001 |
| FMPE (spectral) | 0.365 ± 0.005 | 0.165 ± 0.004 | 0.091 ± 0.002 | 0.075 ± 0.002 |

### Put everything together with svg utils

In [ ]:
base_path = "ice_plots/"
kwargs_text = {"size": "7pt", "font": "Arial", "weight": "800"}


# create new SVG figure
fig = sg.SVGFigure("14cm", "10cm")

# load matpotlib-generated figures
fig0 = sg.fromfile(base_path + "ice_transect_v2.svg")
fig1 = sg.fromfile(base_path + "post_predictives.svg")

fig2 = sg.fromfile(base_path + "results_ice.svg")


# get the plot objects
plot0 = fig0.getroot()
plot1 = fig1.getroot()
plot2 = fig2.getroot()

# get sizes
size0 = get_size_tuple(fig0)
size1 = get_size_tuple(fig1)
size2 = get_size_tuple(fig2)

# define scales
scales = [0.19, 1, 1]

# a: ice transect
plot0.scale(scales[0])
plot0.moveto(10, 15)

# b: post predictive
plot1.scale(scales[1])
plot1.moveto(size0[0] * scales[0] + 20, 10)

# c: results
plot2.scale(scales[2])
plot2.moveto(0, size0[1] * scales[0] + 45)

# add text labels
#txt0 = sg.TextElement(1, 10, "a", **kwargs_text)
#txt1 = sg.TextElement(size0[0] * scales[0] + 22, 10, "b", **kwargs_text)
#txt2 = sg.TextElement(1, size0[1] * scales[0] + 40, "c", **kwargs_text)
#txt3 = sg.TextElement(1, size0[1] * scales[0] + 40, "d", **kwargs_text)
#txt4 = sg.TextElement(1, size0[1] * scales[0] + 40, "e", **kwargs_text)

txt0 = sg.TextElement(-3, 10, "a", **kwargs_text)
txt1 = sg.TextElement(size0[0] * scales[0] + 22, 10, "b", **kwargs_text)
txt2 = sg.TextElement(-3, size0[1] * scales[0] + 45, "c", **kwargs_text)
txt3 = sg.TextElement(115, size0[1] * scales[0] + 45, "d", **kwargs_text)
txt4 = sg.TextElement(235, size0[1] * scales[0] + 45, "e", **kwargs_text)


# append plots and labels to figure
fig.append(
    [
        plot0,
        plot1,
        plot2,
    ]
)
fig.append([txt0, txt1, txt2, txt3, txt4])


# save generated SVG files
fig.save("ice_plots/ice_joint.svg")